In [1]:
pip install tigramite

  Using cached tigramite-5.2.1.30-py3-none-any.whl (250 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
! mamba install -y pynco pynio pyngl


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.1.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pynco', 'pynio', 'pyngl']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━━━━╸━━━━━━━━━━━━━━━╸━   0.0 B 

In [3]:
import gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cftime
from nco import Nco
from tqdm import tqdm
import Ngl #plotting science data
import xesmf as xe

import numpy as np
import matplotlib.pyplot as plt
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import parcorr, gpdc, cmiknn

from utils.constants import TAU_MIN, TAU_MAX
import utils.links as links


In [4]:
fs = gcsfs.GCSFileSystem()
fs.ls("gs://leap-persistent-ro/sungdukyu")

['leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.grid-info.zarr',
 'leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.input.zarr',
 'leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.output.zarr',
 'leap-persistent-ro/sungdukyu/testing']

In [5]:
mapper_in = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.input.zarr')
mapper_out = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.output.zarr')
mapper_grid = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.grid-info.zarr')

ds_train_in = xr.open_dataset(mapper_in, engine='zarr')
ds_train_out = xr.open_dataset(mapper_out, engine='zarr')
gridpoints = xr.open_dataset(mapper_grid, engine ='zarr')

## Reformat the Data in Time of Day

In [6]:
# see above: **ymd** includes date information: the first digit indicates the index of year, the next two digits indicate the month and the last three digits indicates the calendar day in the year.
year=ds_train_in['ymd']//10000 # gets first digit
month=ds_train_in['ymd']%10000//100 # gets 4 right digits then gets the 2 left digits
day=ds_train_in['ymd']%10000%100 # gets 4 right digits then gets the 2 right digits
hour=ds_train_in['tod']//3600 # seconds per minute, minutes per hour
minute=ds_train_in['tod']%3600//60 # removes hours, leaves number of seconds in the day, then divide by 60 to get minutes

t = []
for k in range(len(ds_train_in['ymd'])): # go through ds
    t.append(cftime.DatetimeNoLeap(year[k],month[k],day[k],hour[k],minute[k]))
ds_train_in['sample'] = t # Add the time array to the 'sample' dimension and rename
ds_train_in = ds_train_in.rename({'sample':'time'})
ds_train_in = ds_train_in.drop(['tod','ymd'])

ds_train_in.time.values[0:5] # Check the current **time** dimension, read the timestep

array([cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 20, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 40, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 20, 0, 0, has_year_zero=True)],
      dtype=object)

In [7]:
# see above: **ymd** includes date information: the first digit indicates the index of year, the next two digits indicate the month and the last three digits indicates the calendar day in the year.
year=ds_train_out['ymd']//10000 # gets first digit
month=ds_train_out['ymd']%10000//100 # gets 4 right digits then gets the 2 left digits
day=ds_train_out['ymd']%10000%100 # gets 4 right digits then gets the 2 right digits
hour=ds_train_out['tod']//3600 # seconds per minute, minutes per hour
minute=ds_train_out['tod']%3600//60 # removes hours, leaves number of seconds in the day, then divide by 60 to get minutes

t = []
for k in range(len(ds_train_out['ymd'])): # go through ds
    t.append(cftime.DatetimeNoLeap(year[k],month[k],day[k],hour[k],minute[k]))
ds_train_out['sample'] = t # Add the time array to the 'sample' dimension and rename
ds_train_out =ds_train_out.rename({'sample':'time'})
ds_train_out = ds_train_out.drop(['tod','ymd'])

ds_train_out.time.values[0:5] # Check the current **time** dimension, read the timestep

array([cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 20, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 40, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 20, 0, 0, has_year_zero=True)],
      dtype=object)

In [8]:
ds_train_in.time.values[0:5]

array([cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 20, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 40, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 1, 20, 0, 0, has_year_zero=True)],
      dtype=object)

In [9]:
optional_ozone = 'pbuf_ozone'
predictor_names = ['pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX', 'state_ps', 'state_q0001', 'state_t']
predictand_names = ['cam_out_PRECC']

ds_predictors = ds_train_in.isel(lev=30, ncol=0)[predictor_names]
ds_predictands = ds_train_out.isel(lev=30, ncol=0)[predictand_names]
ds_predictors


<xarray.Dataset>
Dimensions:      (time: 210240)
Coordinates:
  * time         (time) object 0001-02-01 00:00:00 ... 0009-01-31 23:40:00
Data variables:
    pbuf_SOLIN   (time) float64 ...
    pbuf_LHFLX   (time) float64 ...
    pbuf_SHFLX   (time) float64 ...
    state_ps     (time) float64 ...
    state_q0001  (time) float64 ...
    state_t      (time) float64 ...
Attributes:
    calendar:  NO_LEAP
    fv_nphys:  2
    ne:        4

In [19]:
# Suppose 'da1' and 'da2' are your xarray DataArrays
combined = xr.merge([ds_predictors, ds_predictands])

In [12]:
# Convert the Dataset to a DataArray
da = combined.to_array(dim='variable')
da

<xarray.DataArray (variable: 7, time: 210240)>
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.94887745e+01,  8.81729726e+01,  8.68632561e+01, ...,
         1.62621733e+01,  1.64821702e+01,  1.64541293e+01],
       [ 5.65951016e+00,  5.66661830e+00,  5.70201366e+00, ...,
        -8.70921640e-01, -7.95356733e-01, -6.66897434e-01],
       ...,
       [ 2.26157023e-04,  2.27636397e-04,  2.29254597e-04, ...,
         2.75018489e-04,  2.76115305e-04,  2.76252896e-04],
       [ 2.43663599e+02,  2.43657356e+02,  2.43656510e+02, ...,
         2.45024436e+02,  2.45026154e+02,  2.45023641e+02],
       [ 2.94767248e-12,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])
Coordinates:
  * time      (time) object 0001-02-01 00:00:00 ... 0009-01-31 23:40:00
  * variable  (variable) object 'pbuf_SOLIN' 'pbuf_LHFLX' ... 'cam_out_PRECC'
Attributes:
    calendar:  NO_LEAP
    fv_nphys:  2
    ne:        4

In [22]:
da.time

<xarray.DataArray 'time' (time: 210240)>
array([cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 20, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 40, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(9, 1, 31, 23, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(9, 1, 31, 23, 20, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(9, 1, 31, 23, 40, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 0001-02-01 00:00:00 ... 0009-01-31 23:40:00

In [17]:
da.T.shape

(210240, 7)

In [23]:
dataframe = pp.DataFrame(da.T.values,
                         datatime=da.time.values,
                         var_names=predictor_names.extend(predictand_names))

ParCorr = parcorr.ParCorr(significance='analytic')
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=ParCorr, verbosity=1)


TypeError: In analysis mode 'single'. 'data' is of type <class 'xarray.core.dataarray.DataArray'>, must be np.ndarray or dict.

In [15]:
pip install dcor

  Using cached dcor-0.6-py3-none-any.whl (55 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def load_vars_xarray(input_vars, output_vars, downsample=True, chunks = False):
    # raw files, not interpolated according to Yu suggestion
    if(chunks):
        mapper = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.input.zarr')
        inp = xr.open_dataset(mapper, engine='zarr', chunks={})
        mapper = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.output.zarr')
        output = xr.open_dataset(mapper, engine='zarr', chunks={})
    else:
        mapper = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.input.zarr')
        inp = xr.open_dataset(mapper, engine='zarr')
        mapper = fs.get_mapper('leap-persistent-ro/sungdukyu/E3SM-MMF_ne4.train.output.zarr')
        output = xr.open_dataset(mapper, engine='zarr')
    if downsample: # might as well do first
        inp = inp.isel(sample = np.arange(36,len(inp.sample),72)) #  every 1 day
        output = output.isel(sample = np.arange(36,len(output.sample),72))
    ds = inp[input_vars]
    for var in output_vars:
        ds['out_'+var] = output[var]
    time = pd.DataFrame({"ymd":inp.ymd, "tod":inp.tod})
    # rename sample to reformatted time column 
    f = lambda ymd, tod : cftime.DatetimeNoLeap(ymd//10000, ymd%10000//100, ymd%10000%100, tod // 3600, tod%3600 // 60)
    time = time.apply(lambda x: f(x.ymd, x.tod), axis=1)
    ds['sample'] = list(time)
    ds = ds.rename({'sample':'time'})
    ds = ds.assign_coords({'ncol' : ds.ncol})
    return(ds)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import parcorr, gpdc, cmiknn

from utils.constants import TAU_MIN, TAU_MAX
import utils.links as links

def dyn(x1,x2,x3,x4):
    x1 = -0.287*x2 + np.random.normal()
    x2 = 0.4*x2 + 0.287*x1+ np.random.normal()
    x3 = 0.9*x3 + np.random.normal()
    x4 = 0.9 * x2 + np.random.normal()
    return x1,x2,x3,x4


x1_ini,x2_ini,x3_ini,x4_ini = np.random.rand(4,1) #initial state
timestep = 200
data = np.expand_dims(np.concatenate((x1_ini,x2_ini,x3_ini,x4_ini ), axis=0),axis=0) #shape = [1,4]

for step in range(ds_train_in): # run dynamics
    x1_ini,x2_ini,x3_ini,x4_ini = dyn(x1_ini,x2_ini,x3_ini,x4_ini) # ds_train_in
    temp = np.expand_dims(np.concatenate((x1_ini,x2_ini,x3_ini,x4_ini), axis=0),axis=0) #(1,4)
    data = np.concatenate((data, temp), axis=0)  #add to data variable
print(data.shape) #shape = [timestep+1,4]
var_names = ['x1', 'x2', 'x3', 'x4']
dataframe = pp.DataFrame(data,
                         datatime=np.arange(len(data)),
                         var_names=predictor_names)

spcam_data, var_names, parents, children = links.prepare_tigramite_data(list_var_data)

selected_links = links.select_links(TAU_MIN, TAU_MAX, parents, children)

# Initialize dataframe object, specify time axis and variable names
dataframe = pp.DataFrame(spcam_data, datatime=np.arange(len(spcam_data)), var_names=var_names)

pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test, verbosity=verbosity)